In [ ]:
from cyschoolhousesuite import *
from simple_cysh import *
import xlwings as xw

In [ ]:
## Q2 ToT Freeze
# df = get_report(report_key = "00O1a000002rhjq")
# df['Intervention Session Date'] = pd.to_datetime(df['Intervention Session Date'])
# df = df[df['Intervention Session Date']<'02/02/2018']
# df = df.groupby(['Student Reference ID', 'Program: Program Name'])[['Amount of Time (in minutes)']].sum()
# df = df.reset_index()
# df.to_excel("C:\\Users\\CLuedtke\\Q2 ToT Freeze.xlsx")

In [ ]:
logf = open("C:\\Users\\City_Year\\Desktop\\ToT_Audit.log", "w")
logf.write("{0} --- starting script {1}\n".format(str(pd.to_datetime('now')), "'SY18 ToT Audit'"))

sch_ref = "Z:\\ChiPrivate\\Chicago Data and Evaluation\\SY18\\SY18 Team Placement\\SY18 FINAL Team Placements.xlsx"
sch_ref_df = pd.read_excel(sch_ref, sheetname="School Data")
sch_ref_df = sch_ref_df.loc[pd.notnull(sch_ref_df['School'])]

ToT_audit_dir = "Z:\\ChiPrivate\\Chicago Reports and Evaluation\\SY18\\SY18 ToT Audit\\"
ToT_audit_tmplt = "Z:\\ChiPrivate\\Chicago Reports and Evaluation\\SY18\\Eval Management\\SY18 ToT Audit Template.xlsx"

# Read in all ToT Audits to one dataframe
all_tot_audits = []

for x in sch_ref_df["cyconnect Folder"]:
    tot_audit_path = "Z:\\" + x + "\\SY18 ToT Audit - " + sch_ref_df['Informal School Name'][sch_ref_df['cyconnect Folder'] == x].values[0] + ".xlsx"
    try:
        sch_tot_audit = pd.read_excel(tot_audit_path)
        sch_tot_audit = sch_tot_audit.loc[pd.notnull(sch_tot_audit['School'])]
        all_tot_audits.append(sch_tot_audit)
    except Exception as e:
        print(e)
        continue

try:
    all_tot_audits = pd.concat(all_tot_audits, axis = 0)
    all_tot_audits["Action"] = all_tot_audits["Action"].astype(str)
except Exception as e:
    print(e)
    all_tot_audits = pd.DataFrame(columns=["Time ID", "Action"])
    pass

# Select ToT instances to delete and write to safe space
prev_tot2del = pd.read_excel(ToT_audit_dir + "ToT instances to delete.xlsx")
tot2del = all_tot_audits.loc[all_tot_audits["Action"] == "Nothing, this session will be deleted."]

# Delete them
for ISR_name in tot2del['Time ID']:
    try:
        ISR_id = cysh.Intervention_Session_Result__c.get_by_custom_id('Name', ISR_name)['Id']
        cysh.Intervention_Session_Result__c.delete(ISR_id)
    except Exception as e:
        print(e)
        pass

tot2del["z_Date_Added"] = pd.to_datetime('now')
tot2del = prev_tot2del.append(tot2del)
tot2del.to_excel(ToT_audit_dir + "ToT instances to delete.xlsx", index=False, freeze_panes=(1,0))

# Select ToT instances to keep and write to safe space
prev_tot2keep = pd.read_excel(ToT_audit_dir + "ToT instances to keep.xlsx")
tot2keep = all_tot_audits.loc[all_tot_audits["Action"] == "Coach the ACM to have shorter sessions and to log each session separately."]
tot2keep["z_Date_Added"] = pd.to_datetime('now')
tot2keep = prev_tot2keep.append(tot2keep)
tot2keep.to_excel(ToT_audit_dir + "ToT instances to keep.xlsx", index=False, freeze_panes=(1,0))

df = get_report(report_key = "00O1a000002RyU6")

cols = ['School: Account Name',
        'Intervention Session Date',        
        'Intervention Primary Staff: Staff Name',
        "Related Student's Name",
        'Program: Program Name',
        'In School/ Extended Learning',
        'Amount of Time (in minutes)',
        'Time ID',
        'Inter. Session ID']

df = df[cols]

df.loc[df['Program: Program Name'] == "Tutoring: Math", 'Program: Program Name'] = "Math"
df.loc[df['Program: Program Name'] == "Tutoring: Literacy", 'Program: Program Name'] = "ELA"
df.loc[df['Program: Program Name'] == "SEL Check In Check Out", 'Program: Program Name'] = "SEL"

df['Intervention Session Date'] = pd.to_datetime(df['Intervention Session Date'])

# Remove ToT instances in tot2keep and tot2del from new report
df = df.loc[~df["Time ID"].isin(tot2del["Time ID"]) & ~df["Time ID"].isin(tot2keep["Time ID"])]

# Add in processing information for items that carry over week to week (like if a session was known to be a typo, but the mistake wasn't fixed yet)
df = df.merge(right=all_tot_audits[["Time ID", 'Result of ACM Check-in   [EDIT]']], how='left', on="Time ID")

df.sort_values(by=['School: Account Name', 
                   "Intervention Primary Staff: Staff Name", 
                   "Intervention Session Date"], ascending=True, inplace=True)

counts = df.groupby(['School: Account Name']).size()
counts.to_excel(ToT_audit_dir + "ToT Error Counts by School.xlsx", header=False)

In [ ]:
# Open Excel template to fill in data and re-distribute
wb = xw.Book(ToT_audit_tmplt)
sht = wb.sheets['ToT Audit']
 
for x in sch_ref_df["School"]:
    try:
        sht.range('A2:J500').clear_contents()

        # Write ToT Audit data
        sht.range('A2').options(index=False, header=False).value = df[df['School: Account Name'] == x]

        wb.save("Z:\\" 
                + sch_ref_df['cyconnect Folder'][sch_ref_df['School'] == x].values[0]
                + "\\SY18 ToT Audit - "
                + sch_ref_df['Informal School Name'][sch_ref_df['School'] == x].values[0]
                + ".xlsx")
        logf.write("{0} - success\n".format(str(x)))
        
    except Exception as e:
        logf.write(f"{x} - FAIL: {e}\n")
        continue

wb.close()
logf.write("Completed script 'SY18 ToT Audit'\n")
logf.close()

In [ ]:
counts.sort_values(ascending=False)